In [38]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
sns.set()

In [39]:
debug = False
folder = os.path.join("..", "..","resultsVM", "regressionMicro")
outputFile = "microbenchRegressionAll.csv"

In [40]:
def load_microbench_csv(path: str) -> pd.DataFrame:
    """load csv and do some basic preprocess(some benchmark names are broken)
    """
    

    df: pd.DataFrame = pd.read_csv(path,header=None,
                        names=[
                            "suiteRun",
                            "baseline",
                            "path",
                            "name",
                            "version",
                            "invocations",
                            "runtime"],
                    sep=";")
    

    df = df[['name', 'suiteRun', 'path', 'version','runtime','invocations']]

    df.sort_values(by=["name"], inplace=True)


    df.loc[(df["name"] == "16") & (df["runtime"] < 2500), "name"] = "BenchmarkStorageAddRows1"
    df.loc[(df["name"] == "16") & (df["runtime"] < 10000), "name"] = "BenchmarkStorageAddRows2"
    df.loc[(df["name"] == "16") & (df["runtime"] < 100000), "name"] = "BenchmarkStorageAddRows3"
    df.loc[(df["name"] == "16"), "name"] = "BenchmarkStorageAddRows4"

    

    
    df = df.assign(suiteRun=df['suiteRun'].str.extract(r"(?P<suiteRun>\d)-[\d]-[\d]", expand=False))
    df = df.assign(name=df['name'].str.strip('\"'))
    
    df['version'] = df['version'].apply(pd.to_numeric)
    df['suiteRun'] = df['suiteRun'].apply(pd.to_numeric)
    
    return df


def aggregate_data(df: pd.DataFrame) -> list:
    group: pd.DataFrame
    instanceRun = df['instanceRun'].sample().values
    errorCounter = [[0 for x in range(3)] for y in range(2)]
    

    aggr = []
    groupBy: pd.core.groupby.DataFrameGroupBy = df.groupby(["suiteRun", "path", "name"])
    for names, group in groupBy:
        if debug:
            print(f" Aggregating values for benchmark benchmark{names[2]} with path{names[1]} in run {names[0]}")     

        df1 = group[group['version'] == 1]
        df2 = group[group['version'] == 2]
        #There must be 5 values in each dataframe (5 iterations)
        error = False
        if not len(df1) == 5:
            print(f"  There are {len(df1)} elements for version 1, "
            f"experimentRun: {instanceRun}, suiteRun: {names[0]}, commitNumber: {commitNumber}, "
            f"path: {names[1]}, name: {names[2]}")
            error = True
            errorCounter[0][int(names[0])] += 1
        if not len(df2) == 5:
            print(f"  There are {len(df2)} elements for version 2, "
            f"experimentRun: {instanceRun}, suiteRun: {names[0]}, commitNumber: {commitNumber}, "
            f"path: {names[1]}, name: {names[2]}")
            error = True
            errorCounter[1][int(names[0])] += 1
        #Find median
        if not error:
            aggr.append(group)

    print(f"  Found {len(groupBy)} benchmarks")
    print(f"  Errorcounter ([version1], [version2]): {errorCounter}")


    return aggr

In [41]:
aggr = []

all_runs = glob.glob(os.path.join(folder, "run[0-9]"))
for fullRunFolder in all_runs:
    all_numbers = glob.glob(os.path.join(fullRunFolder, "*"))
    for fullNumberFolder in all_numbers:

        all_files = glob.glob(os.path.join(fullNumberFolder, "*"))
        size = len(all_files)

        if size < 7:
            print(f"only {size} elements in folder {fullNumberFolder}, skip.")
        else:
            tmp = fullNumberFolder.split(os.sep)

            #Parse run and commit number from path structure
            instanceRun = tmp[-2][-1]
            commitNumber = tmp[-1]
            print(f"Instance Run {instanceRun} Number {commitNumber} ...")

            #Read latency file for both types (old and new)
            microbenchs = load_microbench_csv(os.path.join(fullNumberFolder, "microbenchResults.csv"))
            microbenchs['instanceRun'] = instanceRun
            microbenchs['number'] = commitNumber
            
            aggr.extend(aggregate_data(microbenchs))


aggr = pd.concat(aggr)


Instance Run 1 Number 51 ...
  Found 549 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 66 ...
  Found 549 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 46 ...
  Found 549 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 56 ...
  Found 549 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 61 ...
  Found 549 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 36 ...
  Found 549 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 11 ...
  Found 549 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 16 ...
  There are 0 elements for version 1, experimentRun: ['1'], suiteRun: 0, commitNumber: 16, path: /lib/persistentqueue/persistentqueue_timing_test.go/BenchmarkQ

In [44]:
# Convert to data frame
df_detail = aggr
df_detail.sort_values(by=["name","number", "instanceRun","suiteRun"], inplace=True)
df_detail.describe()

,suiteRun,version,runtime,invocations
count,230580.000000,230580.000000,2.305800e+05,2.305800e+05
mean,1.000000,1.500000,3.991259e+07,2.433912e+07
std,0.816498,0.500001,2.239190e+08,1.270227e+08
min,0.000000,1.000000,1.253000e+00,1.000000e+00
25%,0.000000,1.000000,5.972000e+03,7.570000e+02
50%,1.000000,1.500000,3.914450e+04,3.063200e+04
75%,2.000000,2.000000,1.564030e+06,2.008155e+05
max,2.000000,2.000000,5.197470e+09,9.573467e+08


In [45]:
df_detail.to_csv(outputFile, sep=";")
